In [1]:
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [2]:
import gc
gc.collect()

42

In [3]:
# torch.cuda.empty_cache()

In [4]:
class Opt:
    def __init__(self):
        self.n_epochs = 200
        self.batch_size = 54 # 128*128 img_size*img_size
        self.lr = 0.0002
        self.b1 = 0.5
        self.b2 = 0.999
        self.n_cpu = 8
        self.latent_dim  = 62
    #    self.n_classes = 10
        self.img_size = 128
        self.channels = 3
        self.sample_interval = 400

In [5]:
opt = Opt()

In [6]:
img_shape = (opt.channels, opt.img_size, opt.img_size)

cuda = True if torch.cuda.is_available() else False


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity


# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

In [7]:
# https://discuss.pytorch.org/t/error-expected-more-than-1-value-per-channel-when-training/26274
test = generator.eval()
test = discriminator.eval()

In [8]:
p = r"C:\Users\原神\Downloads\cats"+"\\"
name = "cats"

In [9]:
# import os, sys
# from PIL import Image

# size = (128,128)

# for infile in os.listdir(p+"val\\"+name):
#         im = Image.open(p+"val\\"+name+"\\"+infile)
#         im = im.resize(size)
#         im = im.convert("RGB")
#         os.remove(p+"val\\"+name+"\\"+infile)
#         im.save(p+"val\\"+name+"\\"+infile.replace("png","jpg"), "JPEG")

# for infile in os.listdir(p+"train\\"+name):
#         im = Image.open(p+"train\\"+name+"\\"+infile)
#         im = im.resize(size)
#         im = im.convert("RGB")
#         os.remove(p+"train\\"+name+"\\"+infile)
#         im.save(p+"train\\"+name+"\\"+infile.replace("png","jpg"), "JPEG")

In [10]:
dataset = datasets.ImageFolder(root=p,
                               transform = transforms.Compose(
                                   [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]),
                              )

In [11]:
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=opt.batch_size,
    shuffle=True,
)

In [12]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [13]:
opt.n_epochs = 5000

In [14]:
# optimizer_G.param_groups[0]['lr'] = 1e-4
# optimizer_D.param_groups[0]['lr'] = 1e-4

In [15]:
opt.sample_interval = 100

In [16]:
batches_done = 0

In [17]:
for epoch in range(opt.n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done += 1
        if batches_done % opt.sample_interval == 0:
            save_image(gen_imgs.data[:25], r"C:\Users\原神\Downloads\result1"+"\\%d.png" % batches_done, nrow=5, normalize=True)

C:\Users\原神\AppData\Local\Temp\ipykernel_2884\1089692113.py:5: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:80.)
  valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)


[Epoch 0/5000] [Batch 0/1] [D loss: 0.690324] [G loss: 0.693300]
[Epoch 1/5000] [Batch 0/1] [D loss: 0.599729] [G loss: 0.698724]
[Epoch 2/5000] [Batch 0/1] [D loss: 0.358436] [G loss: 0.709128]
[Epoch 3/5000] [Batch 0/1] [D loss: 0.337673] [G loss: 0.723888]
[Epoch 4/5000] [Batch 0/1] [D loss: 0.327015] [G loss: 0.740180]
[Epoch 5/5000] [Batch 0/1] [D loss: 0.319749] [G loss: 0.754252]
[Epoch 6/5000] [Batch 0/1] [D loss: 0.309037] [G loss: 0.778558]
[Epoch 7/5000] [Batch 0/1] [D loss: 0.303055] [G loss: 0.795050]
[Epoch 8/5000] [Batch 0/1] [D loss: 0.281394] [G loss: 0.860987]
[Epoch 9/5000] [Batch 0/1] [D loss: 0.258833] [G loss: 0.979224]
[Epoch 10/5000] [Batch 0/1] [D loss: 0.278586] [G loss: 0.892772]
[Epoch 11/5000] [Batch 0/1] [D loss: 0.217008] [G loss: 1.108344]
[Epoch 12/5000] [Batch 0/1] [D loss: 0.189435] [G loss: 1.288124]
[Epoch 13/5000] [Batch 0/1] [D loss: 0.196934] [G loss: 1.152338]
[Epoch 14/5000] [Batch 0/1] [D loss: 0.193045] [G loss: 1.194804]
[Epoch 15/5000] [Bat

KeyboardInterrupt: 

In [21]:
for i in range(1,20):
    save_image(gen_imgs.data[-i], r"C:\Users\原神\Downloads\result1"+"\\%d.png" % (i-1), nrow=1, normalize=True)

In [ ]:
#ffmpeg -framerate 8 -i %d.png -crf 0 -pix_fmt yuva420p logo.mp4

In [18]:
# img_embeddings

In [19]:
# gen_imgs

In [20]:
# generator.l1(z)

In [21]:
# generator.l1(z)

In [22]:
        # gen_imgs = generator(z)
        # recon_imgs, img_embeddings = discriminator(gen_imgs)

In [23]:
# z

In [24]:
# gen_imgs

In [21]:
img_embeddings.size()

torch.Size([19, 32])

In [26]:
# pullaway_loss(img_embeddings)

In [27]:
# pixelwise_loss(recon_imgs, gen_imgs.detach())

In [28]:
# embeddings = img_embeddings
# norm = torch.sqrt(torch.sum(embeddings ** 2, -1, keepdim=True))
# norm

In [29]:
# normalized_emb = embeddings / norm
# normalized_emb

In [30]:
# similarity = torch.matmul(normalized_emb, normalized_emb.transpose(1, 0))
# similarity

In [31]:
# batch_size = embeddings.size(0)
# batch_size

In [32]:
# (torch.sum(similarity) - batch_size)

In [33]:
# (batch_size * (batch_size - 1))

In [34]:
# batch_size

In [37]:
# generator.init_size

In [36]:
# opt.img_size // 4

In [37]:
# z

In [38]:
# h = nn.Sequential(nn.Linear(opt.latent_dim, 128 * 32 ** 2))
# h.cuda()
# h(z)

In [39]:
# generator.l1 = nn.Sequential(nn.Linear(opt.latent_dim, 128 * generator.init_size ** 2))
# generator.l1.cuda()

In [40]:
# g = Generator()
# g.cuda()

In [42]:
# g.l1(z)

In [43]:
# g.eval()

In [44]:
# g.apply(weights_init_normal)

In [45]:
# generator.l1.cuda()

In [46]:
# generator.l1(z)

In [47]:
# generator.l1(z)

In [48]:
# generator.__call__

In [49]:
# generator(z)

In [50]:
# opt.latent_dim

In [51]:
# imgs.shape

In [52]:
# np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))

In [53]:
# -6.47948121e-01

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# sample_image(n_row=opt.n_classes, batches_done=batches_done)

In [ ]:
gp = r"C:\Users\TEST\generator_001.pth"
dp = r"C:\Users\TEST\discriminator_001.pth"

In [ ]:
torch.save(generator.state_dict(), gp)
torch.save(discriminator.state_dict(), dp)

In [ ]:
#generate a random image
sample_image(n_row=1, batches_done=-1)

In [ ]:
discriminator.load_state_dict(torch.load(dp, weights_only=True))
discriminator.eval()

In [ ]:
generator.load_state_dict(torch.load(gp, weights_only=True))
generator.eval()